In [5]:
import sqlite3
import pandas as pd

In [2]:
connect = sqlite3.connect("a.db")
cursor = connect.cursor()

In [3]:
cursor.execute("create table meetings(start_time varchar, end_time varchar)")
cursor.execute("insert into meetings values('08:00', '09:15')")
cursor.execute("insert into meetings values('13:20', '15:20')")
cursor.execute("insert into meetings values('10:00', '14:00')")
cursor.execute("insert into meetings values('13:55', '16:25')")
cursor.execute("insert into meetings values('14:00', '17:45')")
cursor.execute("insert into meetings values('14:05', '17:45')")
connect.commit()

In [11]:
def f(sql_command, connect=connect):
    df = pd.read_sql(sql_command, connect)
    print(df)

In [14]:
f("select * from meetings")

  start_time end_time
0      08:00    09:15
1      13:20    15:20
2      10:00    14:00
3      13:55    16:25
4      14:00    17:45
5      14:05    17:45


In [28]:
f("""
SELECT start_time AS time_point, 1 AS enter_exit
FROM meetings
UNION ALL
SELECT end_time AS time_point, -1 AS enter_exit
FROM meetings
ORDER BY time_point
""")

   time_point  enter_exit
0       08:00           1
1       09:15          -1
2       10:00           1
3       13:20           1
4       13:55           1
5       14:00           1
6       14:00          -1
7       14:05           1
8       15:20          -1
9       16:25          -1
10      17:45          -1
11      17:45          -1


In [43]:
f("""
SELECT enter_exit,SUM(enter_exit) OVER (ORDER BY (SELECT NULL)) AS cumulative_sum FROM(
    SELECT start_time AS time_point, 1 AS enter_exit
    FROM meetings
    UNION ALL
    SELECT end_time AS time_point, -1 AS enter_exit
    FROM meetings
    ORDER BY time_point
)
""")

    enter_exit  cumulative_sum
0            1               0
1           -1               0
2            1               0
3            1               0
4            1               0
5            1               0
6           -1               0
7            1               0
8           -1               0
9           -1               0
10          -1               0
11          -1               0


In [52]:
f("""
SELECT MAX(Cumulative) as rooms FROM(
    SELECT enter_exit,SUM(enter_exit) OVER (ORDER BY idx) as Cumulative FROM(
        SELECT ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS idx, enter_exit FROM(
            SELECT start_time AS time_point, 1 AS enter_exit
            FROM meetings
            UNION ALL
            SELECT end_time AS time_point, -1 AS enter_exit
            FROM meetings
            ORDER BY time_point
        )
    )
)
""")

   rooms
0      4


In [16]:
# CHATGPT solution
s = """
SELECT COUNT(*) AS max_overlap
FROM (
    SELECT time_point, 
           SUM(enter_exit) AS active_intervals
    FROM (
        SELECT start_time AS time_point, 1 AS enter_exit
        FROM meetings
        UNION ALL
        SELECT end_time AS time_point, -1 AS enter_exit
        FROM meetings
    ) AS time_events
    GROUP BY time_point
    ORDER BY time_point
) AS overlap_count;
"""

f(s)

   max_overlap
0           10


In [22]:
# CHATGPT solution
def max_overlapping_intervals(intervals):
    events = []
    for start, end in intervals:
        events.append((start, 1))  # 1 indicates a start event
        events.append((end, -1))   # -1 indicates an end event

    events.sort() # Sort events by position and type # key=lambda x: (x[0], -x[1])
    print(events)

    max_overlap = 0
    current_overlap = 0
    for _, event_type in events:
        current_overlap += event_type
        max_overlap = max(max_overlap, current_overlap)
        print((current_overlap, max_overlap), end=", ")

    return max_overlap

# Example usage
intervals = [(1, 5), (2, 6), (3, 7), (4, 8), (6,9), (2,10), (-2,10)]
result = max_overlapping_intervals(intervals)
print("Number of maximum overlapped intervals:", result)

[(-2, 1), (1, 1), (2, 1), (2, 1), (3, 1), (4, 1), (5, -1), (6, 1), (6, -1), (7, -1), (8, -1), (9, -1), (10, -1), (10, -1)]
(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (5, 6), (6, 6), (5, 6), (4, 6), (3, 6), (2, 6), (1, 6), (0, 6), Number of maximum overlapped intervals: 6
